In [23]:
import pandas as pd
import plotly.express as px
import streamlit as st
import plotly.graph_objects as go
import numpy as np

In [4]:
df_kronyx = pd.read_csv("../data/Justin.csv")
df_gotwiller = pd.read_csv("../data/Gautier.csv")

# 2. Aperçu rapide
df_kronyx.head()      

,uts,utc_time,artist,artist_mbid,album,album_mbid,track,track_mbid
0,1759297343,"01 Oct 2025, 05:42",HUNTR/X,NaN,KPop Demon Hunters (Soundtrack from the Netfli...,17395f62-3bc4-4afd-9584-15506d51fdd1,Golden,f40cee59-ae8b-45cf-98e8-9c7278dbd821
1,1759297204,"01 Oct 2025, 05:40",Derik Fein,NaN,Hollywood,NaN,Hollywood,NaN
2,1759297023,"01 Oct 2025, 05:37",Sabrina Carpenter,1882fe91-cdd9-49c9-9956-8e06a3810bd4,Singular Act II,26d0c26d-1f74-45fd-a139-f90323d851aa,Looking at Me,bf20e876-1b34-42df-9db7-81054f4aac1a
3,1759296861,"01 Oct 2025, 05:34",Grace Power,NaN,Siren Song,NaN,Siren Song,NaN
4,1759263010,"30 Sep 2025, 20:10",Nessa Barrett,da0c8f4f-f4f4-4e91-86ca-e31108bb5706,dying on the inside,NaN,dying on the inside,eea59ae1-ad45-47f3-b5b1-145523c7b28f


In [5]:
df_gotwiller.head()  

,uts,utc_time,artist,artist_mbid,album,album_mbid,track,track_mbid
0,1756792590,"02 Sep 2025, 05:56",eyeto8,NaN,Boy Meets the World,NaN,Sound of Time Passing,NaN
1,1756792013,"02 Sep 2025, 05:46",eyeto8,NaN,The Old Electronic Eldorado,NaN,The Shift,NaN
2,1756791598,"02 Sep 2025, 05:39",eyeto8,NaN,The Old Electronic Eldorado,NaN,Audentity,NaN
3,1756791295,"02 Sep 2025, 05:34",eyeto8,NaN,The Old Electronic Eldorado,NaN,Urgent,NaN
4,1756705616,"01 Sep 2025, 05:46",eyeto8,NaN,The Old Electronic Eldorado,NaN,Lost Disco Thrills,NaN


In [6]:
for df in [df_kronyx, df_gotwiller]:
    df["utc_time"] = pd.to_datetime(df["utc_time"], format="%d %b %Y, %H:%M")
    # Ajouter des colonnes utiles
    df["date"] = df["utc_time"].dt.date
    df["hour"] = df["utc_time"].dt.hour
    df["weekday"] = df["utc_time"].dt.day_name()
    df["minute"] = df["utc_time"].dt.minute

In [7]:
df_kronyx.head()

,uts,utc_time,artist,artist_mbid,album,album_mbid,track,track_mbid,date,hour,weekday,minute
0,1759297343,2025-10-01 05:42:00,HUNTR/X,NaN,KPop Demon Hunters (Soundtrack from the Netfli...,17395f62-3bc4-4afd-9584-15506d51fdd1,Golden,f40cee59-ae8b-45cf-98e8-9c7278dbd821,2025-10-01,5,Wednesday,42
1,1759297204,2025-10-01 05:40:00,Derik Fein,NaN,Hollywood,NaN,Hollywood,NaN,2025-10-01,5,Wednesday,40
2,1759297023,2025-10-01 05:37:00,Sabrina Carpenter,1882fe91-cdd9-49c9-9956-8e06a3810bd4,Singular Act II,26d0c26d-1f74-45fd-a139-f90323d851aa,Looking at Me,bf20e876-1b34-42df-9db7-81054f4aac1a,2025-10-01,5,Wednesday,37
3,1759296861,2025-10-01 05:34:00,Grace Power,NaN,Siren Song,NaN,Siren Song,NaN,2025-10-01,5,Wednesday,34
4,1759263010,2025-09-30 20:10:00,Nessa Barrett,da0c8f4f-f4f4-4e91-86ca-e31108bb5706,dying on the inside,NaN,dying on the inside,eea59ae1-ad45-47f3-b5b1-145523c7b28f,2025-09-30,20,Tuesday,10


## Visualisations

In [49]:
# --- Charger les données ---
df1 = pd.read_csv("../data/Justin.csv")
df2 = pd.read_csv("../data/Gautier.csv")
time_min = max(df1["uts"].min(), df2["uts"].min())
df1["user"] = "justin"
df2["user"] = "gautier"

df = pd.concat([df1, df2], ignore_index=True)
df= df[df["uts"] > time_min]
# --- Convertir la colonne de temps ---
df["utc_time"] = pd.to_datetime(df["utc_time"], format="%d %b %Y, %H:%M")

# --- Colonnes utiles ---
df["date"] = df["utc_time"].dt.date
df["year"] = df["utc_time"].dt.year
df["hour"] = df["utc_time"].dt.hour
df["weekday"] = df["utc_time"].dt.day_name()
df["week"] = df["utc_time"].dt.isocalendar().week


In [50]:
time_min
#1739983023

1739983023

In [53]:

# --- Sélecteur d'année ---
annees_disponibles = sorted(df["year"].unique())
year_selected = st.selectbox("Année à analyser", annees_disponibles)

# Filtrer l'année sélectionnée
df_year = df[df["year"] == year_selected]

jours_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

# 1. Regrouper par semaine / jour / utilisateur
comp = (
    df_year.groupby(["week", "weekday", "user"])
           .size()
           .reset_index(name="plays")
)

# 2. Pivot table → weekday × week × user
pivot = comp.pivot_table(
    index=["weekday", "week"],
    columns="user",
    values="plays",
    fill_value=0
).reset_index()

pivot.head()

# 3. Calcul de qui a le plus écouté
pivot["compare"] = pivot["justin"] - pivot["gautier"]

2025-11-21 11:25:47.740 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-21 11:25:47.742 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-21 11:25:47.744 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-21 11:25:47.746 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-21 11:25:47.747 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-21 11:25:47.748 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-21 11:25:47.749 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [56]:
pivot.head()

user,weekday,week,gautier,justin,compare,compare_flag
0,Friday,8,67.0,76.0,9.0,1
1,Friday,9,93.0,39.0,-54.0,-1
2,Friday,10,4.0,49.0,45.0,1
3,Friday,11,8.0,7.0,-1.0,-1
4,Friday,12,30.0,14.0,-16.0,-1


In [59]:
# -1 : Gautier gagne / 0 : égalité / +1 : Justin gagne
pivot["compare_flag"] = pivot["compare"].apply(lambda x:
    1 if x > 0 else (-1 if x < 0 else 0)
)

# 4. Matrice pour la heatmap
matrix_compare = pivot.pivot(
    index="weekday",
    columns="week",
    values="compare_flag"
).reindex(jours_order)

# 5. Remap des valeurs en [0,1] pour Plotly
# -1 → 0 (jaune)
#  0 → 0.5 (gris)
# +1 → 1 (bleu)
matrix_plot = (matrix_compare + 1) / 2

# --- Labels axes ---
semaines = [f"W{w}" for w in matrix_compare.columns]
jours = matrix_compare.index.tolist()


fig = px.imshow(
    matrix_plot,
    x=semaines,
    y=jours,
    color_continuous_scale=["yellow", "lightgray", "blue"],
    aspect="auto",
)

fig.update_traces(
    hovertemplate="%{y}, %{x}<extra></extra>"
)

fig.update_layout(
    title=f"Qui écoute le plus ? — Année {year_selected}",
    xaxis_title="Semaine",
    yaxis_title="Jour de la semaine",
    plot_bgcolor='white',
    paper_bgcolor='white'
)
fig.show()
st.plotly_chart(fig)


2025-11-21 11:32:05.881 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-21 11:32:05.882 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-21 11:32:05.883 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-21 11:32:05.884 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-21 11:32:05.885 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


DeltaGenerator()